<a href="https://colab.research.google.com/github/Rohit-515/tts_finetuned_model/blob/main/speechT5_finetune_hindi/implementation_and_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
with open('requirements.txt', 'w') as f:
    f.write('numpy==1.23.5\n')
    f.write('transformers\n')
    f.write('datasets\n')
    f.write('soundfile\n')
    f.write('torch\n')
    f.write('torchaudio\n')
    f.write('sentencepiece\n')
    f.write('speechbrain==0.5.16\n')
    f.write('librosa\n')
    f.write('gradio==4.44.1\n')

In [4]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [1]:
import gradio as gr
import torch
import soundfile as sf
import os
import numpy as np
import re
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from speechbrain.pretrained import EncoderClassifier
from datasets import load_dataset

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
def load_models_and_data():
    model_name = "microsoft/speecht5_tts"
    processor = SpeechT5Processor.from_pretrained(model_name)
    model = SpeechT5ForTextToSpeech.from_pretrained("rohit0619/speecht5_finetuned_rohit_hindi").to(device)
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(device)

    spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
    speaker_model = EncoderClassifier.from_hparams(
        source=spk_model_name,
        run_opts={"device": device},
        savedir=os.path.join("/tmp", spk_model_name),
    )

    # Load a sample from a dataset for default embedding
    dataset = load_dataset("1rsh/tts-rj-hi-karya", split="train")
    example = dataset[10]

    return model, processor, vocoder, speaker_model, example

model, processor, vocoder, speaker_model, default_example = load_models_and_data()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer 

README.md:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

train-00000-of-00015.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

train-00001-of-00015.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

train-00002-of-00015.parquet:   0%|          | 0.00/493M [00:00<?, ?B/s]

...

train-00013-of-00015.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

train-00014-of-00015.parquet:   0%|          | 0.00/495M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/75.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/422603 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4269 [00:00<?, ? examples/s]

In [6]:
def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform).unsqueeze(0).to(device))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze()
    return speaker_embeddings

def prepare_default_embedding(example):
    audio = example["audio"]
    return create_speaker_embedding(audio["array"])

default_embedding = prepare_default_embedding(default_example)


In [7]:
replacements = {
    ('ँ', 'n'),   # Anusvara (nasal sound)
    ('ं', 'n'),   # Anusvara (nasal sound)
    ('ः', 'h'),   # Visarga (aspirated sound)
    ('अ', 'uh'),
    ('आ', 'aa'),
    ('इ', 'i'),
    ('ई', 'ee'),
    ('उ', 'u'),
    ('ऊ', 'oo'),
    ('ऋ', 'ri'),
    ('ऍ', 'ae'),
    ('ए', 'e'),
    ('ऐ', 'ai'),
    ('ऑ', 'aw'),
    ('ओ', 'o'),
    ('औ', 'au'),
    ('क', 'k'),
    ('ख', 'kh'),
    ('ग', 'g'),
    ('घ', 'gh'),
    ('च', 'ch'),
    ('छ', 'chh'),
    ('ज', 'j'),
    ('झ', 'jh'),
    ('ञ', 'ny'),
    ('ट', 't'),
    ('ठ', 'th'),
    ('ड', 'd'),
    ('ढ', 'dh'),
    ('ण', 'n'),
    ('त', 't'),
    ('थ', 'th'),
    ('द', 'd'),
    ('ध', 'dh'),
    ('न', 'n'),
    ('प', 'p'),
    ('फ', 'ph'),
    ('ब', 'b'),
    ('भ', 'bh'),
    ('म', 'm'),
    ('य', 'y'),
    ('र', 'r'),
    ('ल', 'l'),
    ('व', 'v'),
    ('श', 'sh'),
    ('ष', 'shh'),
    ('स', 's'),
    ('ह', 'h'),
    ('़', ''),    # Nukta (diacritic mark for foreign sounds)
    ('ा', 'aa'),  # Vowel sound modifier
    ('ि', 'i'),   # Vowel sound modifier
    ('ी', 'ee'),  # Vowel sound modifier
    ('ु', 'u'),   # Vowel sound modifier
    ('ू', 'oo'),  # Vowel sound modifier
    ('ृ', 'ri'),  # Vowel sound modifier
    ('े', 'e'),   # Vowel sound modifier
    ('ै', 'ai'),  # Vowel sound modifier
    ('ॉ', 'aw'),  # Vowel sound modifier
    ('ो', 'o'),   # Vowel sound modifier
    ('ौ', 'au'),  # Vowel sound modifier
    ('्', ''),    # Halant (for stopping consonant sound)
    ('क़', 'q'),
    ('ख़', 'kh'),
    ('ग़', 'gh'),
    ('ज़', 'z'),
    ('ड़', 'r'),
    ('ढ़', 'rh'),
    ('फ़', 'f'),
    ('ॠ', 'rri'),
    ('।', 'period'),  # Purnavirama (full stop)
    ('०', '0'),
    ('१', '1'),
    ('२', '2'),
    ('३', '3'),
    ('४', '4'),
    ('५', '5'),
    ('६', '6'),
    ('७', '7'),
    ('८', '8'),
    ('९', '9')
}

In [8]:
!pip install indic-num2words

from num_to_words import num_to_word

def replace_numbers_with_words(text):
    def replace(match):
        number = int(match.group())
        return number_to_word(number, lang='hi')

    result = re.sub(r'\b\d+\b', replace, text)
    return result


In [11]:
def normalize_text(text):

    # Replace numbers with words
    text = replace_numbers_with_words(text)

    # Apply character replacements
    for old, new in replacements:
        text = text.replace(old, new)

    # Remove punctuation
    text = re.sub(r'[^\u0900-\u097F\s\']', '', text)

    return text

In [13]:
def text_to_speech(text, audio_file=None):
    # Normalize the input text
    normalized_text = normalize_text(text)

    # Prepare the input for the model
    inputs = processor(text=normalized_text, return_tensors="pt").to(device)

    # Use the default speaker embedding
    speaker_embeddings = default_embedding

    # Generate speech
    with torch.no_grad():
        speech = model.generate_speech(inputs["input_ids"], speaker_embeddings.unsqueeze(0), vocoder=vocoder)

    speech_np = speech.cpu().numpy()

    return (16000, speech_np)

In [14]:
iface = gr.Interface(
    fn=text_to_speech,
    inputs=[
        gr.Textbox(label="Enter text to convert to speech")
    ],
    outputs=[
        gr.Audio(label="Generated Speech", type="numpy")
    ],
    title="Hindi Language SpeechT5 Text-to-Speech Demo",
    description="Enter Your text, and listen to the generated speech."
)

In [15]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fcfb20ea2ecfc89815.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##Evaluation of finetuned regional model

The model is not generating anything, model has been trained once again but not getting proper result and further I did not know what to do now

In [ ]:
mos_ratings = 0
naturalness = 0
intelligibility = 0
pronounciation = 0